In [182]:
from bs4 import BeautifulSoup
from datetime import datetime
import feedparser
import json
import numpy as np
import os
import pandas as pd
import requests
import tabula

In [183]:
url = 'https://www.regione.sicilia.it'
feed = feedparser.parse(url+'/feed')

In [184]:
f = [field for field in feed['entries'] if 'bollettino settimanale' in field['title']]
link = f[0]['links'][0]['href']
print(link)

http://www.regione.sicilia.it/la-regione-informa/covid-bollettino-settimanale-aumento-nuovi-casi-calo-prime-dosi


In [185]:
html = requests.get(link)
soup = BeautifulSoup(html.text, 'html.parser')
links = soup.find_all('a')

In [186]:
for link in links:
    if ('.pdf' in link.get('href')):
        pdf = url+link.get('href')

print(pdf)

https://www.regione.sicilia.it/sites/default/files/2021-10/Report%20Completo%2027%20Ottobre%202021.pdf


In [187]:
r = requests.get(pdf, stream=True)
with open('./downloads/'+datetime.now().strftime("%Y%m%d")+'.pdf', 'wb') as f:
    f.write(r.content)
    print("PDF Scaricato.")

PDF Scaricato.


In [201]:
files = []
path = './downloads/'

for file in os.listdir(path):
    if (file.endswith('.pdf')):
        files.append(file)

files.sort()
file = files[0]

## Qui comincia Tabula, ma forse meglio usare OCR

In [271]:
print('Leggo PDF...')
df = tabula.read_pdf_with_template(path+file, 'template_vaccini.json', pandas_options={'header': None}, multiple_tables=False)

Leggo PDF...


Got stderr: Oct 28, 2021 11:54:33 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Oct 28, 2021 11:54:34 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Oct 28, 2021 11:54:34 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Oct 28, 2021 11:54:35 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Oct 28, 2021 11:54:36 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Oct 28, 2021 11:54:36 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Oct 28, 2021 11:54:37 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Oct 28, 2021 11:54:37 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Oct 28, 2021 11:54:38 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Oct 28, 2021 11:54:39 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [265]:

join = pd.concat(df, axis=1)


#print('Converto in CSV.')
#join.to_csv('tabula-'+file[:-4]+'.csv', index=None, header=False)

In [276]:
df[-1]



,Alcamo,"86,63%","83,34%"
0,Buseto Palizzolo,"85,67%","80,54%"
1,Calatafimi-Segesta,"87,87%","85,37%"
2,Campobello di Mazara,"74,95%","70,28%"
3,Castellammare del Golfo,"80,22%","76,17%"
4,Castelvetrano,"78,06%","73,70%"
5,Custonaci,"76,09%","70,00%"
6,Erice,"81,73%","77,06%"
7,Favignana,"82,95%","78,59%"
8,Gibellina,"86,16%","83,21%"
9,Marsala,"84,39%","80,07%"
